# nifti image read, convert to png 

Nifti for convert tool 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data,color
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import numpy as np
from glob import glob
from skimage.io import imread
import nibabel as nib
import imageio
import cv2
from PIL import Image, ImageCms
from skimage.util import img_as_ubyte

In [2]:
import json
from glob import glob
import cv2
import skimage, os

def calArea(xs,ys):
    minX=min(xs)
    maxX=max(xs)
    minY=min(ys)
    maxY=max(ys)
    area=(maxY-minY)*(maxX-minX)
    #print(xs)
    #print(ys)
    #print("{} {} {} {} {} {} ALANI :{}".format(maxY,minY,maxX,minX,maxY-minY,maxX-minX,area))
    return area

def choiseLession(cnt,thX):
    cntX=[]
    for item in cnt:
        xs=[]
        ys=[]
        
        #cn=0
        for ix in item:
            xs.append(ix[0][0])
            ys.append(ix[0][1])        
            area=calArea(xs,ys)
        
        if(area>=thX):
            cntX.append([xs,ys])
            
    return cntX



def img2jsonAnn(sourcePath,imgPatch,maskSource,maskPatch):
    all_images =sorted(glob(os.path.join(sourcePath,'*'+imgPatch+'*')))
    all_masks  =sorted(glob(os.path.join(maskSource,'*'+maskPatch+'*')))
    print(len(all_masks))
    data = {}
    i=0
    for i in range(len(all_images)):
        print("{}) {} - {}".format(i,all_images[i],all_masks[i]))
    i=0
    for item in all_masks:
        
        img=all_images[i]
        sizeX=os.stat(img).st_size
        fname=os.path.basename(img)
        fnameA=fname.split("_")
        key=str(fname)+""+str(sizeX)
        
        value={}
        value["fileref"]=""
        value["size"]=sizeX
        value["filename"]=fname
        value["base64_img_data"]=""
        fileattributes={}
        value["fileattributes"]=fileattributes

        regions={}
        shape_attributes={}
        region_attributes={}
        region_attributes["label"]="msMask"
        
        src = cv2.imread(item)
        gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
        blur = cv2.blur(gray, (2, 2))
        ret, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_TOZERO)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        cnt = sorted(contours, key=cv2.contourArea, reverse=True)
                # ROI will be object with biggest contour
        #mask = contours[0]
        #print(len(cnt))
        cntX=choiseLession(cnt,0)
        #print(len(cntX))
        c1=0
        for item in cntX:
            xs=item[0]
            xs.append(xs[0])
            ys=item[1]
            ys.append(ys[0])
            all_points_x=xs
            all_points_y=ys
            dd={}
            shape_attributes={}
            shape_attributes["name"]="polygon"
            xs.append(xs[0])
            ys.append(ys[0])
            shape_attributes["all_points_x"]=str(xs)
            shape_attributes["all_points_y"]=str(ys)
            dd["shape_attributes"]=shape_attributes
            dd["region_attributes"]=region_attributes
            regions[str(c1)]=dd
            c1=c1+1
        value["regions"]=regions
        data[key]=value    
        i=i+1
    json_data = json.dumps(data)

    return json_data

In [ ]:
sourcePath ="D:\\datasets\\lung\\vessel12\\vessel12_secim500\\train"
imgPatch   ="slice"
maskSource ="D:\\datasets\\lung\\vessel12\\vessel12_secim500\\trainMask"
maskPatch  ="mask"
s=img2jsonAnn(sourcePath,imgPatch,maskSource,maskPatch)
print(s)

In [ ]:

filepath="D:\\FW_ilkhasta\\"
BASE_IMG_PATH=os.path.join('..',filepath)
Target_IMG_PATH="D:\\FW_ilkhasta\\"
print(BASE_IMG_PATH)

all_images=sorted(glob(os.path.join(BASE_IMG_PATH,'*Flair*')))
all_masks =sorted(glob(os.path.join(BASE_IMG_PATH,'*msplak*')))
i=0
for item in all_images:
    fname=os.path.basename(all_images[i])
    fnameA=fname.split("_")
    test_image=nib.load(all_images[i]).get_fdata()
    test_mask=nib.load(all_masks[i]).get_fdata()
    i=i+1

    r=test_image.shape[2]
    for yy in range(0,r):
        #print(yy)
        img=test_image[:,:,yy]
        img=np.rot90(img)
        img = img.astype(np.uint8)
        imageName='{}{}_slices_{:03}.png'.format(Target_IMG_PATH,fnameA[0], yy)
        imageio.imwrite(imageName, img)
        print(imageName," saved")
        maskImg=test_mask[:,:,yy]
        maskImg=np.rot90(maskImg)*255
        maskImg = maskImg.astype(np.uint8)
        mimageName='{}{}_slices_{:03}.png'.format(Target_IMG_PATH,fnameA[0], yy)
        imageio.imwrite(mimageName, maskImg)
        print(mimageName," saved")
      
        

In [10]:
filepath="D:\\datasets\\ms\\miccai2008\\flair\\"
BASE_IMG_PATH=os.path.join('..',filepath)
Target_IMG_PATH="D:\\datasets\\ms\\miccai2008\\flairPNG\\"
print(BASE_IMG_PATH)
print(Target_IMG_PATH)

all_images=sorted(glob(os.path.join(BASE_IMG_PATH,'*.nii')))
i=0
for item in all_images:
    fname=os.path.basename(all_images[i])
    fnameA=fname.split(".")[0]
    print(fnameA)
    test_image=nib.load(all_images[i]).get_fdata()
    i=i+1
    r=test_image.shape[2]
    t_img_path=os.path.join(Target_IMG_PATH, fnameA)
    fnameA1=fnameA.split("_")
    parentName="parent_"+fnameA1[0]+"_"+fnameA1[2]
    if not os.path.exists(t_img_path):
        os.makedirs(t_img_path)
    for yy in range(0,r):
        #print(yy)
        img=test_image[:,:,yy]
        if(sum(img)>10):
            img=np.rot90(img)
            img.astype(np.uint8)
            imageName='{}\\{}_slices_{:04}.jpg'.format(t_img_path, parentName, yy)
            imageio.imwrite(imageName, img)
            print(imageName," saved")
    break

Lossy conversion from float64 to uint8. Range [0.0, 209.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 791.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 790.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 812.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 821.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 855.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 900.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 942.0]. Convert image to uint8 prior to saving to suppress this warning.


D:\datasets\ms\miccai2008\flair\
D:\datasets\ms\miccai2008\flairPNG\
CHB_train_Case01_FLAIR
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0041.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0042.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0043.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0044.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0045.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0046.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0047.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0048.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0049.jpg  saved
D:\datasets\ms\mi

Lossy conversion from float64 to uint8. Range [0.0, 1964.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2007.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 1999.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2050.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2188.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2203.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2231.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2211.0]. Convert image to uint8 prior to saving to suppress this w

  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0081.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0082.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0083.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0084.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0085.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0086.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0087.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0088.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0089.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0090.jpg  saved
D:

Lossy conversion from float64 to uint8. Range [0.0, 601.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 549.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 511.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 486.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 451.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 442.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 412.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 389.0]. Convert image to uint8 prior to saving to suppress this warning.


D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0130.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0131.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0132.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0133.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0134.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0135.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0136.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0137.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0138.jpg  saved
D:\datasets\ms\miccai2008\flairPNG\CHB_train_Case01_FLAIR\parent_CHB_Case01_slices_0139.jpg  saved


## maskelere göre dolu olan slicelerı kaydeden bir yapı ayrı ayrı klasörlere kaydeden

In [5]:
filepath="D:\\datasets\\ms\\isbi\\flair\\"
maskpath="D:\\datasets\\ms\\isbi\\mask\\"
BASE_IMG_PATH=os.path.join('..',filepath)
Target_IMG_PATH = "D:\\datasets\\ms\\isbi\\flairPNG2\\"
print(BASE_IMG_PATH)
print(Target_IMG_PATH)

all_images=sorted(glob(os.path.join(BASE_IMG_PATH,'*.nii')))
all_mask_i=sorted(glob(os.path.join(maskpath,'*.nii')))
for item in all_mask_i:

    fname=os.path.basename(item)
    fnameA=fname.split(".")[0]
    print(fnameA)
    test_image=nib.load(item).get_fdata()
    r=test_image.shape[2]
    fnameA1=fnameA.split("_")
    display(fname)
    display(fnameA1)
    parentName="parent_"+fnameA1[0]+"_"+fnameA1[1]
    
    
    imgName=fname.replace("maskl","flair_pp")
    imgPath=os.path.join(filepath,imgName)
    imgMR=nib.load(imgPath).get_fdata()
    print(os.path.exists(os.path.join(filepath,imgName)))
    
    t_img_path=os.path.join(Target_IMG_PATH, fnameA)
    t_mask_path=os.path.join(t_img_path, "GTMask")
    
    if not os.path.exists(t_img_path):
        os.makedirs(t_img_path)
    if not os.path.exists(t_mask_path):
        os.makedirs(t_mask_path)  

    for yy in range(0,r):
        #print(yy)
        img=test_image[:,:,yy]
        imgx=img
        if(imgx.sum()>10):
            img=np.rot90(img)
            img.astype(np.uint8)
            imageName='{}\\{}_GTmask_{:04}.jpg'.format(t_mask_path, parentName, yy)
            imageio.imwrite(imageName, img)
            print(imageName," saved")
            
            img=imgMR[:,:,yy]
            img=np.rot90(img)
            img.astype(np.uint8)
            imageName='{}\\{}_slice_{:04}.jpg'.format(t_img_path, parentName, yy)
            imageio.imwrite(imageName, img)
            print(imageName," saved")
            
    break

D:\datasets\ms\isbi\flair\
D:\datasets\ms\isbi\flairPNG2\
training01_01_mask1


'training01_01_mask1.nii'

['training01', '01', 'mask1']

FileNotFoundError: No such file or no access: 'D:\datasets\ms\isbi\flair\training01_01_mask1.nii'

## maskelere göre dolu olan slicelerı kaydeden bir yapı tek klasörlere kaydeden

In [6]:
filepath="D:\\datasets\\ms\\isbi\\flair\\"
maskpath="D:\\datasets\\ms\\isbi\\mask\\"
Target_IMG_PATH = "D:\\datasets\\ms\\isbi\\flairPNG2\\"
Target_MASK_PATH = "D:\\datasets\\ms\\isbi\\maskPNG2\\"
maskPatch="mask1"
imgPatch="flair_pp"
print(BASE_IMG_PATH)
print(Target_IMG_PATH)

t_img_path=Target_IMG_PATH
t_mask_path=Target_MASK_PATH

all_images=sorted(glob(os.path.join(BASE_IMG_PATH,'*.nii')))
all_mask_i=sorted(glob(os.path.join(maskpath,'*.nii')))
for item in all_mask_i:

    fname=os.path.basename(item)
    fnameA=fname.split(".")[0]
    print(fnameA)
    test_image=nib.load(item).get_fdata()
    r=test_image.shape[2]
    fnameA1=fnameA.split("_")
    parentName="parent_"+fnameA1[0]+"_"+fnameA1[1]
    
    
    imgName=fname.replace(maskPatch,imgPatch)
    imgPath=os.path.join(filepath,imgName)
    imgMR=nib.load(imgPath).get_fdata()
    print(os.path.exists(os.path.join(filepath,imgName)))
    
#     t_img_path=os.path.join(Target_IMG_PATH, fnameA)
#     t_mask_path=os.path.join(t_img_path, "GTMask")
    
#     if not os.path.exists(t_img_path):
#         os.makedirs(t_img_path)
#     if not os.path.exists(t_mask_path):
#         os.makedirs(t_mask_path)  

    for yy in range(0,r):
        #print(yy)
        img=test_image[:,:,yy]
        imgx=img
        if(imgx.sum()>10):
            img=np.rot90(img)
            img.astype(np.uint8)
            imageName='{}\\{}_GTmask_{:04}.jpg'.format(t_mask_path, parentName, yy)
            imageio.imwrite(imageName, img)
            print(imageName," saved")
            
            img=imgMR[:,:,yy]
            img=np.rot90(img)
            img.astype(np.uint8)
            imageName='{}\\{}_slice_{:04}.jpg'.format(t_img_path, parentName, yy)
            imageio.imwrite(imageName, img)
            print(imageName," saved")
            
    

D:\datasets\ms\isbi\flair\
D:\datasets\ms\isbi\flairPNG2\
training01_01_mask1
True


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 41758.921875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 43431.9921875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 44422.55078125]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 46789.4609375]. Convert image to uint8 prior to saving to suppress this w

D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0042.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0042.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0043.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0043.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0044.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0044.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0045.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0045.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0046.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0046.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0047.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0047.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0048.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 45286.83984375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 47557.59765625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 50884.4296875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 57720.62109375]. Convert image to uint8 prior to saving to suppress thi

D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0053.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0053.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0054.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0054.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0055.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0055.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0056.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0056.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0057.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0057.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0058.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training01_01_slice_0058.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training01_01_GTmask_0059.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_

...

  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0081.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0081.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0082.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0082.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0083.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0083.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0084.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0084.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0085.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0085.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0086.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0086.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0087.jpg  saved
D:\datasets\ms\isbi\flairPNG2\

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 900.7427368164062]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 961.033935546875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 955.60693359375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 905.6095581054688]. Convert image to uint8 prior to saving to su

  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0097.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0097.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0098.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0098.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0099.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0099.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0100.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0100.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0101.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0101.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0102.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0102.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0103.jpg  saved
D:\datasets\ms\isbi\flairPNG2\

Lossy conversion from float64 to uint8. Range [0.0, 928.320556640625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 830.2496337890625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 855.38671875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 841.2619018554688]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppr

D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0111.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0111.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0112.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0112.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0113.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0113.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0114.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0114.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0115.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0115.jpg  saved
D:\datasets\ms\isbi\maskPNG2\\parent_training05_04_GTmask_0116.jpg  saved
D:\datasets\ms\isbi\flairPNG2\\parent_training05_04_slice_0116.jpg  saved


In [5]:
img

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import json

def calArea(xs,ys):
    minX=min(xs)
    maxX=max(xs)
    minY=min(ys)
    maxY=max(ys)
    area=(maxY-minY)*(maxX-minX)
    #print(xs)
    #print(ys)
    #print("{} {} {} {} {} {} ALANI :{}".format(maxY,minY,maxX,minX,maxY-minY,maxX-minX,area))
    return area

def choiseLession(cnt,thX):
    cntX=[]
    for item in cnt:
        xs=[]
        ys=[]
        
        #cn=0
        for ix in item:
            xs.append(ix[0][0])
            ys.append(ix[0][1])        
            area=calArea(xs,ys)
        
        if(area>=thX):
            cntX.append([xs,ys])
            
    return cntX
        
data = {}
sizeX=os.stat('D:\\datasets\\3dwmlmr2\\patient01_slices_106_flair.png').st_size
fname=os.path.basename('D:\\datasets\\3dwmlmr2\\patient01_slices_106_flair.png')
fnameA=fname.split("_")
key=str(fname)+""+str(sizeX)
value={}
value["fileref"]=""
value["size"]=sizeX
value["filename"]=fname
value["base64_img_data"]=""
fileattributes={}
value["fileattributes"]=fileattributes

regions={}
shape_attributes={}
region_attributes={}
region_attributes["label"]="msMask"

src = cv2.imread("D:\\datasets\\3dwmlmr2\\patient01_slices_106_GTmask.png")

#print(src)
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
blur = cv2.blur(gray, (3, 3))
# binary thresholding of the image
ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY)
        # ret, thresh = cv2.threshold(gray, 127, 255,0)

        # find contours
        # contours, hierarchy = cv2.findContours(thresh,2,1)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # cc
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
        # ROI will be object with biggest contour
mask = contours[0]
print(len(cnt))

cntX=choiseLession(cnt,50)
print(len(cntX))
c1=0
for item in cntX:
    xs=item[0]
    xs.append(xs[0])
    ys=item[1]
    ys.append(ys[0])
    all_points_x=xs
    all_points_y=ys
    dd={}
    shape_attributes={}
    shape_attributes["name"]="polygon"
    xs.append(xs[0])
    ys.append(ys[0])
    shape_attributes["all_points_x"]=str(xs)
    shape_attributes["all_points_y"]=str(ys)
    dd["shape_attributes"]=shape_attributes
    dd["region_attributes"]=region_attributes
    regions[str(c1)]=dd
    c1=c1+1
    
value["regions"]=regions

data[key]=value    
json_data = json.dumps(data)
print(json_data)


In [ ]:
!jupyter notebook --NotbookApp.iopub_Data_Rate_Limit=1e10

In [ ]:
import cv2
img = cv2.imread('hh.png')
res = cv2.resize(img, dsize=(192, 256), interpolation=cv2.INTER_LANCZOS4)
plt.imshow(res)
imageio.imwrite("hh2.png", res)

img = cv2.imread('maskhh.png')
res = cv2.resize(img, dsize=(192, 256), interpolation=cv2.INTER_LANCZOS4)
plt.imshow(res)
imageio.imwrite("maskhh2.png", res)

In [ ]:
import skimage, os
from glob import glob
import nibabel as nib
import numpy as np
import imageio
import matplotlib.pyplot as plt


filepath="D:\\"
BASE_IMG_PATH=os.path.join('..',filepath)
Target_IMG_PATH="d:\\test\\"
print(BASE_IMG_PATH)

all_images=sorted(glob(os.path.join(BASE_IMG_PATH,'*.nii*')))
i=0
for item in all_images:
    fname=os.path.basename(all_images[i])
    print(fname)
    fnameA=fname.split(".")
    test_image=nib.load(all_images[i]).get_fdata()
    i=i+1

    r=test_image.shape[2]
    print(r)
    for yy in range(0,r):
        print(yy)
        img=test_image[:,:,yy]
        img=np.flip(img,axis=0)
        img=np.rot90(img,1)
        if(yy==13):
            plt.imshow(img)
#             display(img)
            display(np.histogram(img))
#         img = img.astype(np.uint8)
        imageName='{}{}_slices_{:03}.jpg'.format(Target_IMG_PATH,fnameA[0], yy)
        imageio.imwrite(imageName, img)
        print(imageName," saved")


In [ ]:
import dicom2nifti
import dicom2nifti.settings as settings


original_dicom_directory="D:\\datasets\\lung\\ild\\ILD_DB_lungMasksK\\101\\orj\\"
output_file="D:\\sdds.nii.gz"
dicom2nifti.dicom_series_to_nifti(original_dicom_directory, output_file, reorient_nifti=True)

In [ ]:
# reorganization file system 
# ild files seperate each folder
import os
import glob


def fast_scandir(dirname):
    subfolders    = [f.path for f in os.scandir(dirname) if f.is_dir()]
    
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
#     sorted(subfolders) #.sort(key=lambda f: int(filter(str.isdigit, f)))
    return subfolders

def folder2fname(dirname):
    dnX=dirname.split('\\')
    nameX="parent"+dnX[0]
    for d in range(1,len(dnX)):
        nameX+="_"+dnX[d]
    return nameX

root_dir = 'D:\\datasets\\lung\\ild\\ILD_DB_lungMasksK\\'
target_root_dir = 'D:\\datasets\\lung\\ild\\ILD_DB_lungMasksK\\images\\'

fList=fast_scandir(root_dir)
# display(fList)
fListName=[]
for dirname in fList:
    dn=dirname.replace(root_dir, "")
    dnX=folder2fname(dn)
    fListName.append(dnX) 
display(fListName)

    
# file_names = os.listdir(source_dir)
    
# for file_name in file_names:
#     shutil.move(os.path.join(source_dir, file_name), target_dir)

In [ ]:
import cv2
import os
import pydicom
import matplotlib.pyplot as plt
import numpy as np


inputdir = "D:\\datasets\\lung\\ild\\ILD_DB_lungMasksK\\101\\CT-0002-0001.dcm"
outdir = "D:\\datasets\\lung\\ild\\ILD_DB_lungMasksK\\101\\"

ds = pydicom.dcmread(inputdir)
img = ds.pixel_array
display(np.histogram(img))
threshold = 500 # Adjust as needed
img_s = np.array((np.maximum(img, 0) / (np.amax(img) + threshold)) * 255.0, dtype=int)
display(np.histogram(img_s))
cv2.imwrite(outdir +"ff.png" ,img_s)

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob

dicomDic="D:\\datasets\\lung\\ild\\ILD_DB_volumeROIs\\7\\"
# load the DICOM files
files = []
print('glob: {}'.format(dicomDic))
for fname in glob.glob(dicomDic, recursive=False):
    print("loading: {}".format(fname))
    files.append(pydicom.dcmread(fname))

print("file count: {}".format(len(files)))

In [ ]:
from dicom_parser import Image
dicomF="D:\\datasets\\lung\\ild\\ILD_DB_volumeROIs\\7\\CT-7771-0001.dcm"
image = Image(dicomF)

raw_value = image.header.raw['ImagingFrequency'].value
raw_value
type(raw_value)